In [2]:
try:
    import cv2
    import PIL.Image as Image
except:
    %pip install opencv-python
    %pip install pillow
    import cv2
    import PIL.Image as Image
import glob
import os
import numpy as np
from helper.video2frame import video2frame



In [4]:
DATASET_DIR = "needle_dataset/"
PROJECT_DIR = "./"
VIDEO_DIR = "videos"

DATASET_DIR = os.path.abspath(DATASET_DIR)
PROJECT_DIR = os.path.abspath(PROJECT_DIR)
VIDEO_DIR = os.path.abspath(VIDEO_DIR)

print("CHECK OUT YOUR DIRECTORY BY ABSOLUTE PATH")
print("DATASET_DIR:", DATASET_DIR)
print("PROJECT_DIR:", PROJECT_DIR)
print("VIDEO_DIR:", VIDEO_DIR)

CHECK OUT YOUR DIRECTORY BY ABSOLUTE PATH
DATASET_DIR: C:\Project\View\gauge-reading\dataset\needle_dataset
PROJECT_DIR: C:\Project\View\gauge-reading
VIDEO_DIR: C:\Project\View\gauge-reading\videos


In [5]:
FRAME_INTERVAL = 700
TEST_RATIO = 0.1 # 9:1

### Extract frames from video
#### If you are not labeling now, please skip this session

In [15]:
# ALL_VIDEO = True
ALL_VIDEO = True
# If extract_all_video is true,
paths = []
# CAUTION : MAKE SURE NOT TO BE OVERLAPPED. YOUR LABEL IS IMPORTANT
idx_number = 0
CUSTOM_VIDEO = ["1개_3*", "1개_4*"]

if ALL_VIDEO:
    paths = glob.glob(VIDEO_DIR+"/*.mp4")
    VIDEO_TARGET = paths
else:
    for video_path in CUSTOM_VIDEO:
        paths.extend(glob.glob(VIDEO_DIR+video_path))
    print(paths)
    VIDEO_TARGET = paths

if not os.path.isdir(DATASET_DIR+"annotate_here"):
    os.mkdir(DATASET_DIR+"annotate_here")
for video_target in VIDEO_TARGET:
    idx_number = video2frame(video_target, DATASET_DIR+"annotate_here/", FRAME_INTERVAL, idx_number)


700  /  12875  of file  /home/vm01/Project/gauge/videos/one_20cm.mp4  save at  /home/vm01/Project/gauge/dataset/needle_dataset/annotate_here/ 1
1400  /  12875  of file  /home/vm01/Project/gauge/videos/one_20cm.mp4  save at  /home/vm01/Project/gauge/dataset/needle_dataset/annotate_here/ 2
2100  /  12875  of file  /home/vm01/Project/gauge/videos/one_20cm.mp4  save at  /home/vm01/Project/gauge/dataset/needle_dataset/annotate_here/ 3
2800  /  12875  of file  /home/vm01/Project/gauge/videos/one_20cm.mp4  save at  /home/vm01/Project/gauge/dataset/needle_dataset/annotate_here/ 4
3500  /  12875  of file  /home/vm01/Project/gauge/videos/one_20cm.mp4  save at  /home/vm01/Project/gauge/dataset/needle_dataset/annotate_here/ 5
4200  /  12875  of file  /home/vm01/Project/gauge/videos/one_20cm.mp4  save at  /home/vm01/Project/gauge/dataset/needle_dataset/annotate_here/ 6
4900  /  12875  of file  /home/vm01/Project/gauge/videos/one_20cm.mp4  save at  /home/vm01/Project/gauge/dataset/needle_dataset/ann

In [17]:
#PLEASE LABEL ON WINDOWS, OR GUI-BASED UBUNTU
path = DATASET_DIR+"annotate_here/"
%cd $path
try:
    !labelImg
except:
    !pip install labelimg
    !labelImg

#FOR UBUNTU, REFER TO https://pypi.org/project/labelImg/

/home/vm01/Project/gauge/dataset/needle_dataset/annotate_here
QObject::moveToThread: Current thread (0x21038f0) is not the object's thread (0x24b4e80).
Cannot move to target thread (0x21038f0)

qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/home/vm01/Project/gauge/venv/lib/python3.8/site-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb, eglfs, linuxfb, minimal, minimalegl, offscreen, vnc, wayland-egl, wayland, wayland-xcomposite-egl, wayland-xcomposite-glx, webgl.



In [13]:
files = glob.glob(DATASET_DIR+"/annotate_here/*")
from collections import defaultdict
import random
file_num = defaultdict(int)
for _file in files:
    id = _file.split('.')[0]
    file_num[id]+=1

annotated, not_annotated = [], []
for idx, key in enumerate(file_num):
    if file_num[key]%2==1:
        not_annotated.append(key)
    else:
        annotated.append(key)

if len(not_annotated):
    print("CHECK OUT NON_ANNOTATED FILES(MAX 10)")
    for i in range(10):
        if i<len(not_annotated):
            print(not_annotated[i])
print()
print("======================================")
print("Will only process with annotated files")
print("======================================")

# if random.random()<TEST_RATIO:
#     MOVE TO TEST_RATIO
print(annotated)
for key in annotated:
    number = key.split('/')[-1]
    if random.random()<TEST_RATIO:
        os.rename(key+".txt", os.path.join(DATASET_DIR, "labels/test/"+number+".txt"))
        os.rename(key+".jpg", os.path.join(DATASET_DIR, "images/test/"+number+".jpg"))

    else:
        os.rename(key+".txt", os.path.join(DATASET_DIR, "labels/train/"+number+".txt"))
        os.rename(key+".jpg", os.path.join(DATASET_DIR, "images/train/"+number+".jpg"))
print("finished relocation")


CHECK OUT NON_ANNOTATED FILES(MAX 10)
C:\Project\View\gauge-reading\dataset\needle_dataset/annotate_here\042
C:\Project\View\gauge-reading\dataset\needle_dataset/annotate_here\047
C:\Project\View\gauge-reading\dataset\needle_dataset/annotate_here\049
C:\Project\View\gauge-reading\dataset\needle_dataset/annotate_here\051
C:\Project\View\gauge-reading\dataset\needle_dataset/annotate_here\052
C:\Project\View\gauge-reading\dataset\needle_dataset/annotate_here\058
C:\Project\View\gauge-reading\dataset\needle_dataset/annotate_here\059
C:\Project\View\gauge-reading\dataset\needle_dataset/annotate_here\062
C:\Project\View\gauge-reading\dataset\needle_dataset/annotate_here\064
C:\Project\View\gauge-reading\dataset\needle_dataset/annotate_here\065

Will only process with annotated files
['C:\\Project\\View\\gauge-reading\\dataset\\needle_dataset/annotate_here\\014', 'C:\\Project\\View\\gauge-reading\\dataset\\needle_dataset/annotate_here\\015', 'C:\\Project\\View\\gauge-reading\\dataset\\needle_

In [20]:
# IN SOME CASES, \\ (windows) has to be changed to / (linux)
train_data = glob.glob(DATASET_DIR+"/images/train/*.jpg")
train_data = ["train/"+element.split('/')[-1].split('.')[0] for element in train_data]
test_data = glob.glob(DATASET_DIR+"/images/test/*.jpg")
test_data = ["test/"+element.split('/')[-1].split('.')[0] for element in test_data]

In [17]:
# Check train_data

print(len(train_data))

83


In [21]:
# with open(DATASET_DIR+"/train.txt", "w") as f:
#     f.write("\n".join(train_data))
# with open(DATASET_DIR+"/test.txt", "w") as f:
#     f.write("\n".join(test_data))
# print("processing finished")


processing finished
